[Retour au sommaire](../index.ipynb)

# Traitement de données en tables : concaténation et fusion

Nous avons étudié comment filtrer selon les colonnes ou les lignes, mais parfois il est essentiel de fusionner ou de concatener plusieurs données.

## Concaténation avec pandas

Nous allons concaténer les collèges du Finistère avec ceux du Morbihan.

**Problèmes**
- L'encodage des caractères est différent;
- Certaines colonnes du Morbihan sont absentes dans le fichier du Finistère et réciproquement;
- Les colonnes communes des deux fichiers ne portent pas la même dénomination.

**solutions**
- L'**encodage** des caractères ne pose pas de problème, il suffit de le déclarer à l'ouverture du fichier.
- Il est possible de faire **correspondre** le nom des colonnes communes avec un *mapper*.
- Enfin, pour l'absence de certaines colonnes plusieurs choix de **concatenation** s'offre à nous *inner* ou *outer*

In [86]:
import pandas as pd
colleges_morbihan = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/bf9d46b1-5430-4866-ab7d-58a4d794324d", delimiter=';', encoding='iso 8859-15')
# ecriture d'une correspondance entre les dénominations originales et les nouvelles
mapper = {'PATRONYME':'nom_et',
          'VILLE':'commune_et',
          'CODE_POSTAL':'code_postal',
          'ADRESSE':'adresse',
          'STATUT':'statut'}

colleges_morbihan.rename(columns=mapper, inplace=True)
print("La liste des établissements du morbihan")
colleges_morbihan

La liste des établissements du morbihan


,CODE,nom_et,adresse,BP,code_postal,commune_et,CODE_INSEE,statut
0,0561376S,SAINT-HILAIRE,2 RUE DU CHANOINE DREANO,BP 8,56350,ALLAIRE,56001,Privé
1,0561598H,GILLES GAHINET,2 RUE DE CADIC,BP 43,56610,ARRADON,56003,Public
2,0560122D,SAINT-JEAN-BAPTISTE,18 RUE BOURUET-AUBERTOT,BP 5,56610,ARRADON,56003,Privé
3,0560002Y,LE VERGER,14 RUE DU VERGER,NaN,56400,AURAY,56007,Public
4,0561386C,MATHURIN MARTIN,RUE DES ECOLES,NaN,56150,BAUD,56010,Public
...,...,...,...,...,...,...,...,...
85,0560223N,ANTOINE DE SAINT-EXUPERY,1 RUE GUSTAVE COURBET,BP 563,56000,VANNES,56260,Public
86,0560115W,NOTRE-DAME-LE-MENIMUR,85 RUE DE METZ,NaN,56000,VANNES,56260,Privé
87,0560118Z,LE SACRE-COEUR,13 RUE AMIRAL DEFFORGES,NaN,56000,VANNES,56260,Privé
88,0561544Z,SAINT-FRANCOIS-XAVIER,3 RUE THIERS,NaN,56000,VANNES,56260,Privé


Voici la [documentation de la concaténation](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) de la bibliothèque pandas.

In [153]:
import pandas as pd
concatatenation = pd.concat([colleges_morbihan, colleges_finistere],join='inner', ignore_index = True)
concatatenation

,nom_et,adresse,code_postal,commune_et,statut
0,SAINT-HILAIRE,2 RUE DU CHANOINE DREANO,56350.0,ALLAIRE,Privé
1,GILLES GAHINET,2 RUE DE CADIC,56610.0,ARRADON,Public
2,SAINT-JEAN-BAPTISTE,18 RUE BOURUET-AUBERTOT,56610.0,ARRADON,Privé
3,LE VERGER,14 RUE DU VERGER,56400.0,AURAY,Public
4,MATHURIN MARTIN,RUE DES ECOLES,56150.0,BAUD,Public
...,...,...,...,...,...
198,François Tanguy-Prigent,12 rue Jules Ferry - BP 97157,29600.0,SAINT-MARTIN-DES-CHAMPS,Public
199,Les Sables Blancs,Avenue Robert Jan - BP 236,29182.0,CONCARNEAU,Public
200,François Charles,27 rue Pierre Brossolette,29630.0,PLOUGASNOU,Public
201,Coat Mez,BP 30,29460.0,DAOULAS,Public


<div class="alert alert-info">Tester l'autre type de jointure pour la concatenation</div>

## Fusion avec pandas

Nous allons fusionner deux tables : celle des **collèges du Morbihan** avec la table des **géolocalisations des établissements scolaires** français.

Il existe 4 types de fusion possibles.

- inner
- outer
- left
- right

![](img/merge.png)

On télécharge une seule fois le fichier puis on l'ouvre avec pandas.

In [121]:
import os
import pandas as pd
import requests

# le fichier est volumineux, à télécharger une seule fois
local_file = "geolocalisation.csv"
if not local_file in os.listdir():
    url = "https://data.education.gouv.fr/explore/dataset/fr-en-adresse-et-geolocalisation-etablissements-premier-et-second-degre/download/?format=csv&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B"
    data = requests.get(url).content
    with open(local_file, 'wb') as csvfile:
        csvfile.write(data)
        

Limitons le nombre de colonnes dans le fichier de géolocalisations des établissements scolaires (il y en a 35...)

In [122]:
geolocalisations = pd.read_csv(local_file, delimiter = ";")
geolocalisations.drop(axis=1, inplace=True, columns=['Appellation officielle',
                                                     'Code département',
                                                     'Code région',
                                                     'Code académie',
                                                     'Dénomination principale',
                                                     'Patronyme uai',
                                                     'Secteur Public/Privé',
                                                     'Adresse',
                                                     'Lieu dit',
                                                     'Boite postale',
                                                     'Code postal',
                                                     "Localite d'acheminement",
                                                     'Commune',
                                                     'Coordonnee X',
                                                     'Coordonnee Y',
                                                     'EPSG',
                                                     "Qualité d'appariement",
                                                     'Localisation',
                                                     'Code nature',
                                                     'Nature',
                                                     'Code état établissement',
                                                     'Etat établissement',
                                                     'Code commune',
                                                     'Position',
                                                     'secteur_prive_code_type_contrat',
                                                     'secteur_prive_libelle_type_contrat',
                                                     'code_ministere',
                                                     'libelle_ministere', ])
geolocalisations

,Code établissement,Latitude,Longitude,Département,Région,Académie,date_ouverture
0,0310015Y,43.290056,1.065840,Haute-Garonne,Occitanie,Toulouse,1965-05-01
1,0310021E,43.724449,1.583090,Haute-Garonne,Occitanie,Toulouse,1965-05-01
2,0310031R,42.912037,0.693361,Haute-Garonne,Occitanie,Toulouse,1965-05-01
3,0310040A,43.638263,1.462780,Haute-Garonne,Occitanie,Toulouse,1965-05-01
4,0310041B,43.608689,1.440151,Haute-Garonne,Occitanie,Toulouse,1965-05-01
...,...,...,...,...,...,...,...
65426,9870011Y,NaN,NaN,Wallis et Futuna,TOM et Collectivités territoriales,Wallis et Futuna,1975-08-29
65427,9870015C,NaN,NaN,Wallis et Futuna,TOM et Collectivités territoriales,Wallis et Futuna,1985-09-15
65428,9870020H,NaN,NaN,Wallis et Futuna,TOM et Collectivités territoriales,Wallis et Futuna,1988-09-06
65429,9870028S,NaN,NaN,Wallis et Futuna,TOM et Collectivités territoriales,Wallis et Futuna,1992-09-01


Voilà, maintenant nous pouvons **fusionner** les deux tables en se basant sur la clé du code d'établissement (information unique commune aux deux données)

In [128]:
# on nomme la colonne de la même manière que dans le fichier du Morbihan
mapper = {'Code établissement':'CODE'}
geolocalisations.rename(columns=mapper, inplace=True)
fusion = pd.merge(colleges_morbihan, geolocalisations, on=['CODE'], how='left')
fusion

,CODE,nom_et,adresse,BP,code_postal,commune_et,CODE_INSEE,statut,Latitude,Longitude,Département,Région,Académie,date_ouverture
0,0561376S,SAINT-HILAIRE,2 RUE DU CHANOINE DREANO,BP 8,56350,ALLAIRE,56001,Privé,47.635761,-2.165900,Morbihan,Bretagne,Rennes,1973-09-25
1,0561598H,GILLES GAHINET,2 RUE DE CADIC,BP 43,56610,ARRADON,56003,Public,47.626735,-2.831250,Morbihan,Bretagne,Rennes,1985-09-01
2,0560122D,SAINT-JEAN-BAPTISTE,18 RUE BOURUET-AUBERTOT,BP 5,56610,ARRADON,56003,Privé,47.630748,-2.821489,Morbihan,Bretagne,Rennes,1967-04-20
3,0560002Y,LE VERGER,14 RUE DU VERGER,NaN,56400,AURAY,56007,Public,47.662870,-2.985701,Morbihan,Bretagne,Rennes,1967-09-29
4,0561386C,MATHURIN MARTIN,RUE DES ECOLES,NaN,56150,BAUD,56010,Public,47.873746,-3.013520,Morbihan,Bretagne,Rennes,1974-05-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,0560223N,ANTOINE DE SAINT-EXUPERY,1 RUE GUSTAVE COURBET,BP 563,56000,VANNES,56260,Public,47.675757,-2.770421,Morbihan,Bretagne,Rennes,1969-03-18
86,0560115W,NOTRE-DAME-LE-MENIMUR,85 RUE DE METZ,NaN,56000,VANNES,56260,Privé,47.667811,-2.765060,Morbihan,Bretagne,Rennes,1967-04-12
87,0560118Z,LE SACRE-COEUR,13 RUE AMIRAL DEFFORGES,NaN,56000,VANNES,56260,Privé,47.653512,-2.773351,Morbihan,Bretagne,Rennes,1967-04-12
88,0561544Z,SAINT-FRANCOIS-XAVIER,3 RUE THIERS,NaN,56000,VANNES,56260,Privé,47.654035,-2.760170,Morbihan,Bretagne,Rennes,1980-09-01


<div class="alert alert-info">
Tester et comprendre les 4 types de fusions possibles
</div>

In [135]:
fusion.to_csv('fusion.csv', sep=';')

In [149]:
left = pd.DataFrame(
    {
        "Prénom": ["Alan", "Ada", "Richard", "Linus"],
        "Nom": ["Turing", "Lovelace", "Stallman", "Thorwalds"],
        "Age": [22, 32, 65, 55],
        "Classe": ["1ere2", "1ere1", "1ere2", "1ere3"],
    }
)
left

,Prénom,Nom,Age,Classe
0,Alan,Turing,22,1ere2
1,Ada,Lovelace,32,1ere1
2,Richard,Stallman,65,1ere2
3,Linus,Thorwalds,55,1ere3


In [150]:
right = pd.DataFrame(
    {
        "Prénom": ["Alan", "John", "Richard", "Linus"],
        "Nom": ["Turing", "Von Neumann", "Stallman", "Thorwalds"],
        "Sexe": ["M", "F", "M", "M"],
        "Taille": [183, 175, 180, 185],
    }
)
right

,Prénom,Nom,Sexe,Taille
0,Alan,Turing,M,183
1,John,Von Neumann,F,175
2,Richard,Stallman,M,180
3,Linus,Thorwalds,M,185


In [158]:
merge = pd.merge(left, right, on=['Prénom', 'Nom'], how='outer')
merge

,Prénom,Nom,Age,Classe,Sexe,Taille
0,Alan,Turing,22.0,1ere2,M,183.0
1,Ada,Lovelace,32.0,1ere1,NaN,NaN
2,Richard,Stallman,65.0,1ere2,M,180.0
3,Linus,Thorwalds,55.0,1ere3,M,185.0
4,John,Von Neumann,NaN,NaN,F,175.0


<table style="width:100%">
    <tr>
        <td style="text-align:left;"><a href="3_tri.ipynb"><< 3. Tri des données</a></td>
        <td style="text-align:right;"></td>
    </tr>
</table>

[Retour au sommaire](../index.ipynb)